In [1]:
import tweepy
import csv
import pandas as pd
import os
import json
import datetime
import time
from collections import Counter
import operator

In [2]:
#credentials
api_key = ''
api_key_secret = ''

In [3]:
#get access to tweeter api
auth = tweepy.AppAuthHandler(api_key, api_key_secret)
api = tweepy.API(auth)

In [4]:
#get keywords from user
#input must be: word1, word2, .....
print('Enter keywords as folling: word1, word2, ....')
userInput = input()
userInput = userInput.replace(', ', ',').replace(' ,', ',')
keywordsArray = userInput.split(',')
#print(keywordsArray)

Enter keywords as folling: word1, word2, ....


 Jervi


In [5]:
#extra values Helpers
#get current date
lastWeek = datetime.date.today() - datetime.timedelta(weeks=1)
#for Benchmarking the executions
total_found = 0

In [6]:
#consume api and store result in array of 5 columns [key, created_at, text, username, trend]
start_time = time.time()
#storing api result in array
tweetArray = []
trendCollection = [] #collect all posts trends
for key in keywordsArray:
    for tweet in tweepy.Cursor(api.search, q=key).items():
        isEmpty = len(tweet.entities['hashtags']) #check if post has hashtag
        trend = tweet.entities['hashtags'][0]['text'] if isEmpty!=0 else ''    #get first tag only

        created_at = tweet.created_at
        text = tweet.text.encode('utf-8')
        username = tweet.user.screen_name.encode('utf-8')

        tweetArray.append([key, created_at, text, username, trend])
        trendCollection.append(trend)

        total_found = total_found + 1
        
#chrono ends
end_time = time.time()

In [7]:
#prepare Tags array helpers, 
tags = list(set(trendCollection)) #delete duplication
tagWithOccurance = [] #array of tag with its occurence
for tag in tags:
    tagWithOccurance.append([tag, trendCollection.count(tag)])


In [8]:
######## GLOBAL API RESULT ###########

In [9]:
#array of 6 columns with a tagOccurence [key, created_at, text, username, trend, tagOccurence]
finalTweet = []
for item in tweetArray:
    tagName = item[4] #get row tagname
    tagIndex = tags.index(tagName) #get index of the tagName on tag[]
    tagOccurence = tagWithOccurance[tagIndex][1] #get occurence in tagOccurence0-
    finalTweet.append([item[0], item[1], item[2], item[3], item[4], tagOccurence])

In [10]:
#final Tweet sorted by tag occurence
finalTweet.sort(key=lambda x: x[5], reverse=True)

In [11]:
######################################

In [12]:
######## CREATE CSV ##################

In [13]:
#final tweet not sorted default
with open('defaultResults.csv', 'w', newline='', encoding='utf-8') as file:
    filednames = ["keyword", "created_at", "text", "username", "tag", "tag occurence"]
    writer = csv.DictWriter(file, fieldnames=filednames)
    writer.writeheader()
    writer = csv.writer(file, delimiter=',')
    writer.writerows(finalTweet)
    file.close()

In [14]:
#final Tweet sorted by tag occurence
with open('classifiedByOccurence.csv', 'w', newline='', encoding='utf-8') as file:
    filednames = ["keyword", "created_at", "text", "username", "tag", "tag occurence"]
    writer = csv.DictWriter(file, fieldnames=filednames)
    writer.writeheader()
    writer = csv.writer(file, delimiter=',')
    writer.writerows(finalTweet)
    file.close()   

In [15]:
######################################

In [16]:
######## TOP 10 TAGS ################

In [17]:
#----- FUNCTION RESULT --------#

In [18]:
#array to string
def listToString(s):  
    str1 = ""    
    for ele in s:  
        str1 += str(ele)    
    return str1  

In [19]:
#Days tweet
def getTopDayTag(nbDays):
    tagDayCollection_temp = [] #colloect tags of the day
    
    askedDay = datetime.date.today() - datetime.timedelta(days=nbDays)
    dayStart = datetime.datetime(askedDay.year, askedDay.month, askedDay.day, 0)
    dayEnd = datetime.datetime(askedDay.year, askedDay.month, askedDay.day, 23, 59, 59)
    
    for item in finalTweet:
        if item[1] >= dayStart and item[1] <= dayEnd:
            tagDayCollection_temp.append(item[4])
    #today tags
    topDayTag = sorted(set(tagDayCollection_temp), key = lambda ele: tagDayCollection_temp.count(ele), reverse=True) #sort by occurence descendant
    if len(topDayTag) != 0:
        topDayTag.remove('')  #to remove the no tag case
    
    return topDayTag

In [20]:
#Hours Top Tag
def getTopHourTag(nbHours):
    tagHourCollection_temp = [] #colloect tags of this hour

    askedHour = datetime.datetime.today() - datetime.timedelta(hours=nbHours)
    hourStart = datetime.datetime(askedHour.year, askedHour.month, askedHour.day, askedHour.hour, 0)
    hourEnd = datetime.datetime(askedHour.year, askedHour.month, askedHour.day, askedHour.hour, 59)

    for item in finalTweet:
        if item[1] >= hourStart and item[1] <= hourEnd:
            tagHourCollection_temp.append(item[4])

    #hour tags
    topHourTag = sorted(set(tagHourCollection_temp), key = lambda ele: tagHourCollection_temp.count(ele), reverse=True) #sort by occurence descendant
    if len(topHourTag) != 0:
        topHourTag.remove('')  #to remove the no tag case

    return topHourTag

In [21]:
#----- DISPLAY RESULT --------#

In [28]:
#General Details
time_lapsed = format(end_time - start_time, ".4f")
print('\033[4m' + 'Total time spent to get results' + '\033[0m' + ':', end =" ")
print(time_lapsed + ' seconds')
print('\033[4m' + 'Total rows found from api' + '\033[0m' + ':', end =" ")
print(total_found, end=" ")
print('rows')

Total time spent to get results: 8.7486 seconds
Total rows found from api: 128 rows


In [43]:
#Top10 tag week tag
print('Top 10 Hashtags of the week')
topWeekTag = sorted(set(trendCollection), key = lambda ele: trendCollection.count(ele), reverse=True) #sort by occurence descendant
topWeekTag.remove('')  #to remove the no tag case
print(str(topWeekTag[:10]).strip('[]'))


Top 10 Hashtags of the week
'twitch', 'URGENTE', 'PETRI', 'Discord', 'いいねした人の画像欄覗いて素敵絵を私の絵柄で描いてもいいですか'


In [30]:
#Top10 tag days of the week tag
print('Top 10 Hashtags for each day of the week')
for i in range(7):
    askedDay = datetime.date.today() - datetime.timedelta(days=i)
    print(askedDay, end=" ")
    print(':', end=" ")
    result = listToString(getTopDayTag(i)[:10])
    print(result)

Top 10 Hashtags for each day of the week
2021-07-20 : PETRI
2021-07-19 : 
2021-07-18 : 
2021-07-17 : 
2021-07-16 : URGENTE
2021-07-15 : twitch
2021-07-14 : Discord


In [31]:
#Top10 tags per hours of tag
print('Top 10 Hashtags for each hour of today')
print(datetime.date.today())
for i in range(24):
    askedHour = datetime.datetime.today() - datetime.timedelta(hours=i)
    print(askedHour.hour, end=" ")
    print('\t:', end=" ")
    result = listToString(getTopHourTag(i)[:10])
    print(result)

Top 10 Hashtags for each hour of today
2021-07-20
12 	: 
11 	: 
10 	: 
9 	: 
8 	: 
7 	: 
6 	: PETRI
5 	: 
4 	: 
3 	: 
2 	: 
1 	: 
0 	: 
23 	: 
22 	: 
21 	: 
20 	: 
19 	: 
18 	: 
17 	: 
16 	: 
15 	: 
14 	: 
13 	: 
